In [1]:
import pandas as pd
import json
import glob, os
from sfem import SFEM

In [2]:
with open("prisoner_A_log_history.json", "r") as f:
    j = json.load(f)
df = pd.DataFrame(j)

In [3]:
df.columns = ["A", "B"]
df

,A,B
0,cooperate,cooperate
1,defect,cooperate
2,defect,cooperate
3,defect,cooperate
4,defect,cooperate
5,defect,cooperate
6,defect,cooperate
7,defect,cooperate
8,cooperate,cooperate
9,cooperate,cooperate


In [14]:
#defect rate
defect_rate = df['A'][df['A']=="defect"].count()/df['A'].count()

#Reciprocity rate
rr_count = 0
rr_invalid = 0
for row in range(1,df.shape[0]):
    if df['A'].iloc[row] != "None" and df['B'].iloc[row-1] != "None":
        if df['A'].iloc[row] == df['B'].iloc[row-1]:
            rr_count +=1
    else:
        rr_invalid +=1
rr_percentage = rr_count/(df.shape[0]-1)

#Forgiveness rate based off paper "Nicer Than Humans: How do Large Language Models Behave in the Prisoner’s Dilemma?"
f_count = 0
opp_betray_count = 0
penalty = 0
f_invalid = 0
vengeful = False
for row in range(1,df.shape[0]):
    if df['A'].iloc[row] != "None" and df['A'].iloc[row-1] != "None" and df['B'].iloc[row-1] != "None":
        if df['B'].iloc[row-1] == "defect":
            opp_betray_count += 1
            if df['A'].iloc[row] == "cooperate":
                f_count += 1
                vengeful = False
        if row > 1:
            if df['B'].iloc[row-2] != "None":
                if df['B'].iloc[row-2] == "defect" and df['B'].iloc[row-1] == "cooperate":
                    if df['A'].iloc[row] == "defect" and df['A'].iloc[row-1] == "defect":
                        penalty += 1
                        vengeful = True
                    else:
                        vengeful = False
                else:
                    if vengeful and df['A'].iloc[row] == "defect":
                        penalty += 1
                    else:
                        vengeful = False
            else:
                f_invalid +=1
    else:
        f_invalid +=1

if opp_betray_count == 0:
    f_percentage = -1
else:
    f_percentage = f_count / (opp_betray_count + penalty)

#Forgiveness rate based on  we only count forgiveness if the agent plays betrays after opponent betrays and then revert back to cooperation if the opponent seeks forgiveness (switches to cooperates). Penalty term remains.
# f_count = 0
# opp_betray_count = 0
# penalty = 0
# f_invalid = 0
# vengeful = False
# for row in range(1,df.shape[0]):
#     if df['invalid'].iloc[row] == False and df['invalid'].iloc[row-1] == False:
#         if df['betray'].iloc[row-1] == True:
#             opp_betray_count += 1
#         if row > 1:
#             if df['betray'].iloc[row-2] == True and df['betray'].iloc[row-1] == False:
#                 if df['betray'].iloc[row] == True and df['betray'].iloc[row-1] == True:
#                     penalty += 1
#                     vengeful = True
#                 elif df['betray'].iloc[row] == True and df['betray'].iloc[row-1] == False:
#                     f_count += 1
#                     vengeful = False
#         else:
#             if vengeful and df['betray'].iloc[row] == True:
#                 penalty += 1
#             else:
#                 vengeful = False
#     else:
#         f_invalid +=1

# f_percentage = f_count / (opp_betray_count + penalty)

#Exploitability
e_count = 0
opp_betray_count = 0
e_invalid = 0
for row in range(1,df.shape[0]):
    if df['A'].iloc[row] != "None" and df['B'].iloc[row-1] != "None":
        if df['B'].iloc[row-1] == "defect":
            opp_betray_count += 1
            if df['A'].iloc[row] == "cooperate":
                e_count += 1
    else:
        e_invalid +=1
        
if opp_betray_count == 0:
    e_percentage = -1
else:
    e_percentage = e_count / opp_betray_count

#Retaliatory
r_count = 0
opp_betray_count = 0
r_invalid = 0
for row in range(1,df.shape[0]):
    if df['A'].iloc[row] != "None" and df['A'].iloc[row-1] != "None" and df['B'].iloc[row-1] != "None":
        if df['A'].iloc[row-1] == "cooperate" and df['B'].iloc[row-1] == "defect":
            opp_betray_count += 1
            if df['A'].iloc[row] == "defect":
                r_count += 1
    else:
        r_invalid +=1

if opp_betray_count == 0:
    r_percentage = -1
else:
    r_percentage = r_count / opp_betray_count

#Troublemaking
t_count = 0
opportunity_count = 0
t_invalid = 0
for row in range(0,df.shape[0]):
    if row == 0:
        if df['A'].iloc[row] != "None":
            opportunity_count += 1
            if df['A'].iloc[row] == "defect":
                t_count +=1
        else:
            t_invalid +=1 
    else:        
        if df['A'].iloc[row] != "None" and df['B'].iloc[row-1] != "None":
            if df['B'].iloc[row-1] == "cooperate":
                opportunity_count += 1
                if df['A'].iloc[row] == "defect":
                    t_count += 1
        else:
            t_invalid +=1
            
if opportunity_count == 0:
    t_percentage = -1
else:
    t_percentage = t_count / opportunity_count

#SFEM
sfem = SFEM(df)
sfem_res = sfem.save_results()
# Return the result as a dictionary

There are 20 strategies in the strategies.py file. 
The strategies are: ['s01_allc', 's02_alld', 's03_tft', 's04_dtft', 's05_tf2t', 's06_tf3t', 's07_2tft', 's08_2tf2t', 's09_t2', 's10_grim', 's11_grim2', 's12_grim3', 's13_wsls', 's14_2wsls', 's15_ctod', 's16_dtf2t', 's17_dtf3t', 's18_dgrim2', 's19_dgrim3', 's20_dcalt']


/home/coder/miniforge3/envs/repeng/lib/python3.10/site-packages/scipy/optimize/_slsqp_py.py:434: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
/home/coder/miniforge3/envs/repeng/lib/python3.10/site-packages/scipy/optimize/_slsqp_py.py:438: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  g = append(wrapped_grad(x), 0.0)
/home/coder/miniforge3/envs/repeng/lib/python3.10/site-packages/scipy/optimize/_slsqp_py.py:492: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  a_eq = vstack([con['jac'](x, *con['args'])


In [15]:
results = {
        'defect_rate': defect_rate,
        'reciprocity_rate': rr_percentage,
        'reciprocity_invalid': rr_invalid,
        'forgiveness_rate': f_percentage,
        'forgiveness_invalid': f_invalid,
        'exploitability_rate': e_percentage,
        'explotability_invalid': e_invalid,
        'retaliatory_rate': r_percentage,
        'rectaliatory_invalid': r_invalid,
        'troublemaking_rate': t_percentage,
        'troublemaking_invalid': t_invalid,    
        'sfem': sfem_res    
	}

In [16]:
results

{'defect_rate': 0.7,
 'reciprocity_rate': 0.2222222222222222,
 'reciprocity_invalid': 0,
 'forgiveness_rate': -1,
 'forgiveness_invalid': 0,
 'exploitability_rate': -1,
 'explotability_invalid': 0,
 'retaliatory_rate': -1,
 'rectaliatory_invalid': 0,
 'troublemaking_rate': 0.7,
 'troublemaking_invalid': 0,
 'sfem':             Estimates
 beta           0.7000
 s14_2wsls      0.2759
 s13_wsls       0.2461
 s15_ctod       0.2429
 s02_alld       0.2351
 s01_allc       0.0000
 s05_tf2t       0.0000
 s04_dtft       0.0000
 s03_tft        0.0000
 s06_tf3t       0.0000
 s09_t2         0.0000
 s10_grim       0.0000
 s08_2tf2t      0.0000
 s07_2tft       0.0000
 s12_grim3      0.0000
 s11_grim2      0.0000
 s16_dtf2t      0.0000
 s17_dtf3t      0.0000
 s18_dgrim2     0.0000
 s19_dgrim3     0.0000
 s20_dcalt      0.0000
 LL            -6.1086}